<a href="https://colab.research.google.com/github/prabishkhadka/7316/blob/main/supervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import ee
import geemap

In [14]:
#Trigger the authentication file
ee.Authenticate()
#Initilize the library
ee.Initialize(project='class7316')

In [15]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

vis_params = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'gamma': 1.0, 'min': 7734.411922360646, 'max': 11681.179563146603}


In [24]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2016-01-01", "2016-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)

vis_params = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'gamma': 1.0, 'min': 7734.411922360646, 'max': 11681.179563146603}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

In [25]:
ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()

'2016-11-18'

In [26]:
image.get("CLOUD_COVER").getInfo()

0.08

In [28]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
# Map

In [29]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [30]:
print(points.size().getInfo())

3607


In [31]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-121.75363086398976, 37.774777213706344]}, 'id': '0', 'properties': {'landcover': 71}}


In [33]:
type(points)

ee.featurecollection.FeatureCollection

In [52]:
# Use these bands for prediction.


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [41]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'SR_B1': 8010, 'SR_B2': 8218, 'SR_B3': 9872, 'SR_B4': 8660, 'SR_B5': 24065, 'SR_B6': 15464, 'SR_B7': 11414, 'landcover': 71}}


In [42]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)



In [43]:
result.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [7671, 7801],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 462585, 0, -30, 4264215]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-123.3968983753351, 36.803775376222],
    [-123.39689178582451, 36.802491833033486],
    [-123.36734707270641, 36.797086537378355],
    [-122.9121234870415, 36.71297234812813],
    [-121.41795160422467, 36.42331656098203],
    [-121.37223497249072, 36.41414006525577],
    [-121.37041995441417, 36.41411534569524],
    [-121.09339276355736, 37.30862244073357],
    [-121.04531189358332, 37.46202054157521],
    [-120.99212859498824, 37.631223386473174],
    [-120.95302531204044, 37.755209872305464],
    [-120.91153335535509, 37.88641262588368],
    [-120.8909142651011, 37.95146859373793],
    [-120.83703318891044, 38.121281797549834],
    [-1

In [45]:

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")


In [46]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [47]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [48]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [49]:
Map.addLayer(landcover, {}, "Land cover")


In [50]:
Map.add_legend(builtin_legend="NLCD")

In [53]:
ee.batch.Export.image.toDrive(
    image=landcover,
    description="landcover_7316",
    folder="export",
    scale=900
)

<Task Type.EXPORT_IMAGE: landcover_7316 (State.UNSUBMITTED)>